# Clean CIGALE results
Here we join the runs from CIGALE of the three different redsfhit ranges that we use (Low, Intermediate and High). We also present the first figures of this work. First, we import the important packages

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy.table import Table, vstack
from scipy.stats import binned_statistic

In [3]:
# User shortcuts
def perc75(array):
    """75th Percentile"""
    return np.nanpercentile(array, 75)


def perc25(array):
    """25th Percentile"""
    return np.nanpercentile(array, 25)

## Joining CIGALE files

We read the results from the AGN models we are going to use. These models are the ones presented by Stalevski et. al 2012, 2016 [(SKIRTOR)](https://ui.adsabs.harvard.edu/abs/2016MNRAS.458.2288S/abstract) and Fritz et. al 2006 [(Fritz)](https://ui.adsabs.harvard.edu/abs/2006MNRAS.366..767F/abstract).

In [4]:
INTERIM_FOLDER = '../Data/Interim/CIGALEOutputs/'

# SKIRTOR models
ModelSL = Table.read(INTERIM_FOLDER+'results_SKLz_3Dic.fits', format='fits')
ModelS0L = Table.read(INTERIM_FOLDER+'results_SKLz0_3Dic.fits', format='fits')

ModelSI = Table.read(INTERIM_FOLDER+'results_SKIz_3Dic.fits', format='fits')
ModelS0I = Table.read(INTERIM_FOLDER+'results_SKIz0_3Dic.fits', format='fits')

ModelSH = Table.read(INTERIM_FOLDER+'results_SKHz_3Dic.fits', format='fits')
ModelS0H = Table.read(INTERIM_FOLDER+'results_SKHz0_3Dic.fits', format='fits')

# Fritz models
ModelFL = Table.read(INTERIM_FOLDER+'results_FrLz_3Dic.fits', format='fits')
ModelFI = Table.read(INTERIM_FOLDER+'results_FrIz_3Dic.fits', format='fits')
ModelFH = Table.read(INTERIM_FOLDER+'results_FrHz_3Dic.fits', format='fits')

ModelF0 = Table.read(INTERIM_FOLDER+'results_Fr0_3Dic.fits', format='fits')

# Model Two-angles
ModelS37 = Table.read(INTERIM_FOLDER+'results_SK3070_3Dic.fits', format='fits')
ModelF37 = Table.read(INTERIM_FOLDER+'results_Fr3070_3Dic.fits', format='fits')

Then, we stack the tables that where separated for simplicity, and reorganize the list of galaxies so we can compare them with the runs of all the galaxies. Keep in mind that Models_SK0 and Models_Fr0 are practically the same models with some numerical differences (We ran the two different models for consistency and tested that the results are the same).

In [5]:
Models_SK = vstack([ModelSL, ModelSI, ModelSH])
Models_SK0 = vstack([ModelS0L, ModelS0I, ModelS0H])
Models_Fr = vstack([ModelFL, ModelFI, ModelFH])
Models_Fr0 = ModelF0[np.concatenate([np.where(ModelF0['id'] == oid)[0] for oid in Models_Fr['id']])]
Models_S37 = ModelS37[np.concatenate([np.where(ModelS37['id'] == oid)[0] for oid in Models_SK['id']])]
Models_F37 = ModelF37[np.concatenate([np.where(ModelF37['id'] == oid)[0] for oid in Models_Fr['id']])]

## Checking $\chi_{\rm{red}}^{2}$ distributions
Here we calculate the average (percentiles) values of the different $\chi_{\rm{red}}^{2}$ in the No-AGN and AGN models.
First, with the Fritz models.

In [6]:
Fr0_chis = Models_Fr0['best.reduced_chi_square']
Fr_chis = Models_Fr['best.reduced_chi_square']

print('75%: ', np.log10(perc75((Fr0_chis - Fr_chis))))
print('50%: ', np.log10(np.nanmedian((Fr0_chis - Fr_chis))))
print('25%: ', np.log10(perc25((Fr0_chis - Fr_chis))))

75%:  0.7000196130774164
50%:  0.40621631495356925
25%:  -0.014515131322976474


Then, with the SKIRTOR models

In [7]:
SK0_chis = Models_SK0['best.reduced_chi_square']
SK_chis = Models_SK['best.reduced_chi_square']

print('75%: ', np.log10(perc75((SK0_chis - SK_chis))))
print('50%: ', np.log10(np.nanmedian((SK0_chis - SK_chis))))
print('25%: ', np.log10(perc25((SK0_chis - SK_chis))))

75%:  0.7039671960464844
50%:  0.41391357634130477
25%:  0.0013804643674243068


Then, in average the No-AGN models have a $\chi_{\rm{red}}^{2}$ ~0.41 dex above the AGN models.

##### FIGURE 2

We create an image between $\chi_{\rm{red}}^{2}$ and the $f_{\rm{AGN}}$ to present how important it is to include the AGN in the models. With this figure, we can asses for which galaxies is good to assume a no-AGN model ($f_{\rm{AGN}}=0$). We also present in a separate panel the  $\chi_{\rm{red}}^{2}$  distribution. We calculate the running medians of the trends between $\chi_{\rm{red}}^{2}$ and the $f_{\rm{AGN}}$.

In [8]:
bin_means, bin_edges, binN = binned_statistic(Models_SK['bayes.agn.fracAGN'],
                                              Models_SK['best.reduced_chi_square'],
                                              statistic=np.nanmedian,
                                              bins=np.linspace(0.1, 0.9, 17))
bin_means2, bin_edges2, biN2 = binned_statistic(Models_SK['bayes.agn.fracAGN'],
                                                Models_SK0['best.reduced_chi_square'],
                                                statistic=np.nanmedian,
                                                bins=np.linspace(0.1, 0.9, 17))
bin_means3, bin_edges3, binN3 = binned_statistic(Models_Fr['bayes.agn.fracAGN'],
                                                 Models_Fr['best.reduced_chi_square'],
                                                 statistic=np.nanmedian,
                                                 bins=np.linspace(0.1, 0.9, 17))
bin_means4, bin_edges4, binN4 = binned_statistic(Models_Fr['bayes.agn.fracAGN'],
                                                 Models_Fr0['best.reduced_chi_square'],
                                                 statistic=np.nanmedian,
                                                 bins=np.linspace(0.1, 0.9, 17))
binsChiS = np.linspace(-2.0, 2.5, 46)  # Assuming Chisquare bins in this ranges

Here the values for Fritz and SKIRTOR No-AGN models change due to a different $f_{\rm{AGN}}$ in the models. Then, we plot the FIGURE 2.

In [9]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6.75, 9.44),
                                   sharex=True, gridspec_kw={'height_ratios': [1, 3]})
    ax2.scatter(np.log10(Models_SK['best.reduced_chi_square']),
                Models_SK['bayes.agn.fracAGN'], alpha=0.02)
    ax2.scatter(np.log10(Models_SK0['best.reduced_chi_square']),
                Models_SK['bayes.agn.fracAGN'], alpha=0.02)
    ax2.scatter(np.log10(Models_Fr['best.reduced_chi_square']),
                Models_Fr['bayes.agn.fracAGN'], alpha=0.02)
    ax2.scatter(np.log10(Models_Fr0['best.reduced_chi_square']),
                Models_Fr['bayes.agn.fracAGN'], alpha=0.02)
    ax2.vlines(np.log10(bin_means), bin_edges[:-1], bin_edges[1:], colors='C0', lw=3,
               label='SKIRTOR')
    ax2.vlines(np.log10(bin_means2), bin_edges2[:-1], bin_edges2[1:], colors='C1', lw=3,
               label='SKIRTOR No-AGN')
    ax2.vlines(np.log10(bin_means3), bin_edges3[:-1], bin_edges3[1:], colors='C2', lw=3,
               label='Fritz')
    ax2.vlines(np.log10(bin_means4), bin_edges4[:-1], bin_edges4[1:], colors='C3', lw=3,
               label='Fritz No-AGN')
    ax2.axvline(x=0.5, ls='--', color='gray')
    ax2.axvline(x=-0.5, ls='--', color='gray')
    ax2.set_xlabel(r'$\log(\chi^2_{\rm{red}})$')
    ax2.set_ylabel('AGN Fraction')
    ax2.legend(loc='best')
    ax1.set_xlabel('AGN Fraction')
    ax1.hist(np.log10(Models_SK['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='SKIRTOR', lw=2)
    ax1.hist(np.log10(Models_SK0['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='SKIRTOR No-AGN', lw=2)
    ax1.hist(np.log10(Models_Fr['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='Fritz', lw=2)
    ax1.hist(np.log10(Models_Fr0['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='Fritz No-AGN', lw=2)
    ax1.axvline(x=0.5, ls='--', color='gray')
    ax1.axvline(x=-0.5, ls='--', color='gray')
    ax1.set_ylabel('# Galaxies')
    ax1.set_xlim(-2, 2)
    ax2.set_ylim(0.08, 0.92)
    plt.subplots_adjust(wspace=0, hspace=0)
#     plt.savefig('../Figures/F2.pdf',dpi=300,bbox_inches = 'tight') # For Paper
#     plt.savefig('../Talk_Figures/F2.png',dpi=300,bbox_inches = 'tight') # For Presentation

## Cleaning
We remove all galaxies with:
- Bad fittings: $\chi_{\rm{red}}^{2}$ higher than $10^{0.5}$ (Underestimated) and lower than $10^{-0.5}$ (Overestimated).
- Galaxies where a No-AGN is probable using the Bayesian inference criterion (BIC)
- Galaxies with a bad constrained SFR

For the second condition we need to know the number of bands we are using for each galaxy. We retrieve that number from the original photometry files that we use to run CIGALE.

In [10]:
PhotL = Table.read('../Data/Interim/CIGPhot_EnergyBal_Lowz.tbl',
                   format='ascii')
PhotI = Table.read('../Data/Interim/CIGPhot_EnergyBal_Intz.tbl',
                   format='ascii')
PhotH = Table.read('../Data/Interim/CIGPhot_EnergyBal_Highz.tbl',
                   format='ascii')
Phot = vstack([PhotL, PhotI, PhotH])
Nbands = [sum(~np.isnan(list(row)[2::2])) for row in Phot]

The Bayesian inference criterion (BIC) is defined as $\rm{BIC}= \chi^2 + k\times\ln(N)$, where k is the number of parameters and N are the number of data points (bands), and is an approximation of the Bayes factor ([Buat et. al 2019](https://ui.adsabs.harvard.edu/abs/2019A%26A...632A..79B/abstract), [Salmon et al. 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...827...20S/abstract)). Then, we can calculate the $\Delta\rm{BIC}$ to compare the No-AGN and the AGN models.

In [11]:
Models_SK['DBIC'] = Models_SK['best.chi_square'] - Models_SK0['best.chi_square'] + np.log(Nbands)
Models_Fr['DBIC'] = Models_Fr['best.chi_square'] - Models_Fr0['best.chi_square'] + np.log(Nbands)
Models_S37['DBIC'] = Models_S37['best.chi_square'] - Models_SK0['best.chi_square'] + np.log(Nbands)
Models_F37['DBIC'] = Models_F37['best.chi_square'] - Models_Fr0['best.chi_square'] + np.log(Nbands)

 We adopt a 'positive'(2) significance criteria for the No-AGN models.

In [12]:
GoodSFR_SK = (Models_SK['bayes.sfh.sfr_err'] / (Models_SK['bayes.sfh.sfr']*np.log(10))) <= 1
SClean = Models_SK[np.logical_and.reduce([SK_chis <= 10**0.5, SK_chis >= 10**-0.5,
                                          Models_SK['DBIC'] < 2, GoodSFR_SK])]

GoodSFR_Fr = (Models_Fr['bayes.sfh.sfr_err'] / (Models_Fr['bayes.sfh.sfr']*np.log(10))) <= 1
FClean = Models_Fr[np.logical_and.reduce([Fr_chis <= 10**0.5, Fr_chis >= 10**-0.5,
                                          Models_Fr['DBIC'] < 2, GoodSFR_Fr])]

SK37_chis = Models_S37['best.reduced_chi_square']
GoodSFR_SK37 = (Models_S37['bayes.sfh.sfr_err'] / (Models_S37['bayes.sfh.sfr']*np.log(10))) <= 1
SClean37 = Models_S37[np.logical_and.reduce([SK37_chis <= 10**0.5, SK37_chis >= 10**-0.5,
                                             Models_S37['DBIC'] < 2, GoodSFR_SK37])]

Fr37_chis = Models_F37['best.reduced_chi_square']
GoodSFR_Fr37 = (Models_F37['bayes.sfh.sfr_err'] / (Models_F37['bayes.sfh.sfr']*np.log(10))) <= 1
FClean37 = Models_F37[np.logical_and.reduce([Fr37_chis <= 10**0.5, Fr37_chis >= 10**-0.5,
                                             Models_F37['DBIC'] < 2, GoodSFR_Fr37])]

We additionally print the median values for the simple Fritz and SKIRTOR models.

In [13]:
print('25%: ', 'SK=', perc25(SClean['best.reduced_chi_square']),
      'Fr=', perc25(FClean['best.reduced_chi_square']))
print('50%: ', 'SK=', np.nanmedian(SClean['best.reduced_chi_square']),
      'Fr=', np.nanmedian(FClean['best.reduced_chi_square']))
print('75%: ', 'SK=', perc75(SClean['best.reduced_chi_square']),
      'Fr=', perc75(FClean['best.reduced_chi_square']))

25%:  SK= 0.5995436187684522 Fr= 0.6266459276426563
50%:  SK= 0.9481786461678534 Fr= 0.9805875935858739
75%:  SK= 1.5337985161633152 Fr= 1.5674317430722498


We rename the Fritz models as $i=90-\psi$ (with $\psi$ the angle between the equator to the pole of the system) and the thermal (dust) luminosity to compare with SKIRTOR model.

In [14]:
FClean.rename_column('bayes.agn.psy', 'bayes.agn.i')
FClean.rename_column('bayes.agn.psy_err', 'bayes.agn.i_err')
FClean.rename_column('bayes.agn.therm_luminosity', 'bayes.agn.dust_luminosity')
FClean.rename_column('bayes.agn.therm_luminosity_err',
                     'bayes.agn.dust_luminosity_err')
FClean['bayes.agn.i'] = 90-FClean['bayes.agn.i']

FClean37.rename_column('bayes.agn.psy', 'bayes.agn.i')
FClean37.rename_column('bayes.agn.psy_err', 'bayes.agn.i_err')
FClean37.rename_column('bayes.agn.therm_luminosity',
                       'bayes.agn.dust_luminosity')
FClean37.rename_column('bayes.agn.therm_luminosity_err',
                       'bayes.agn.dust_luminosity_err')
FClean37['bayes.agn.i'] = 90-FClean37['bayes.agn.i']

Finally, we save the files of the cleaned results

In [15]:
SClean.write('../Data/Final/CIGALEOutputs/Cleanresults_SK.fits', format='fits')
FClean.write('../Data/Final/CIGALEOutputs/Cleanresults_Fr.fits', format='fits')
SClean37.write('../Data/Final/CIGALEOutputs/Cleanresults_SK37.fits', format='fits')
FClean37.write('../Data/Final/CIGALEOutputs/Cleanresults_Fr37.fits', format='fits')

## Final numbers and counts by classification type
##### Part TABLE 3
First, we present the final numbers of galaxies that are well-fitted to be analyzed in this work

In [16]:
# For SK models
print('Total number of galaxies:', len(SClean))
Removed_Galaxies = len(Models_SK)-len(SClean)
Cond1 = sum(np.logical_or(SK_chis >= 10**0.5, SK_chis <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_SK['DBIC'] >= 2)/Removed_Galaxies
Cond3 = sum(~GoodSFR_SK)/Removed_Galaxies
print('The fraction of the removed galaxies depending on condition:',
      'Bad Fits=%1.2f, No-AGN=%1.2f and Bad SFR=%1.2f'%(Cond1, Cond2, Cond3))
Cond11 = sum(SK_chis >= 10**0.5)/sum(np.logical_or(SK_chis >= 10**0.5, SK_chis <= 10**-0.5))
Cond12 = sum(SK_chis <= 10**-0.5)/sum(np.logical_or(SK_chis >= 10**0.5, SK_chis <= 10**-0.5))
print('In the Bad Fits the fractions are:',
      'Over-fitting=%1.2f and under-fitting=%1.2f'%(Cond11, Cond12))

Total number of galaxies: 8060
The fraction of the removed galaxies depending on condition: Bad Fits=0.71, No-AGN=0.09 and Bad SFR=0.37
In the Bad Fits the fractions are: Over-fitting=0.51 and under-fitting=0.49


In [17]:
# For Fr models
print('Total number of galaxies:', len(FClean))
Removed_Galaxies = len(Models_Fr)-len(FClean)
Cond1 = sum(np.logical_or(Fr_chis >= 10**0.5, Fr_chis <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_Fr['DBIC'] >= 2)/Removed_Galaxies
Cond3 = sum(~GoodSFR_Fr)/Removed_Galaxies
print('The fraction of the removed galaxies depending on condition:',
      'Bad Fits=%1.2f, No-AGN=%1.2f and Bad SFR=%1.2f'%(Cond1, Cond2, Cond3))
Cond11 = sum(Fr_chis >= 10**0.5)/sum(np.logical_or(Fr_chis >= 10**0.5, Fr_chis <= 10**-0.5))
Cond12 = sum(Fr_chis <= 10**-0.5)/sum(np.logical_or(Fr_chis >= 10**0.5, Fr_chis <= 10**-0.5))
print('In the Bad Fits the fractions are:',
      'Over-fitting=%1.2f and under-fitting=%1.2f'%(Cond11, Cond12))

Total number of galaxies: 8537
The fraction of the removed galaxies depending on condition: Bad Fits=0.75, No-AGN=0.09 and Bad SFR=0.32
In the Bad Fits the fractions are: Over-fitting=0.56 and under-fitting=0.44


In [18]:
# For SK37 models
print('Total number of galaxies:', len(SClean37))
Removed_Galaxies = len(Models_S37)-len(SClean37)
Cond1 = sum(np.logical_or(SK37_chis >= 10**0.5, SK37_chis <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_S37['DBIC'] >= 2)/Removed_Galaxies
Cond3 = sum(~GoodSFR_SK37)/Removed_Galaxies
print('The fraction of the removed galaxies depending on condition:',
      'Bad Fits=%1.2f, No-AGN=%1.2f and Bad SFR=%1.2f'%(Cond1, Cond2, Cond3))
Cond11 = sum(SK37_chis >= 10**0.5)/sum(np.logical_or(SK37_chis >= 10**0.5, SK37_chis <= 10**-0.5))
Cond12 = sum(SK37_chis <= 10**-0.5)/sum(np.logical_or(SK37_chis >= 10**0.5, SK37_chis <= 10**-0.5))
print('In the Bad Fits the fractions are:',
      'Over-fitting=%1.2f and under-fitting=%1.2f'%(Cond11, Cond12))

Total number of galaxies: 8096
The fraction of the removed galaxies depending on condition: Bad Fits=0.70, No-AGN=0.12 and Bad SFR=0.37
In the Bad Fits the fractions are: Over-fitting=0.59 and under-fitting=0.41


In [19]:
# For Fr37 models
print('Total number of galaxies:', len(FClean37))
Removed_Galaxies = len(Models_F37)-len(FClean37)
Cond1 = sum(np.logical_or(Fr37_chis >= 10**0.5, Fr37_chis <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_F37['DBIC'] >= 2)/Removed_Galaxies
Cond3 = sum(~GoodSFR_Fr37)/Removed_Galaxies
print('The fraction of the removed galaxies depending on condition:',
      'Bad Fits=%1.2f, No-AGN=%1.2f and Bad SFR=%1.2f'%(Cond1, Cond2, Cond3))
Cond11 = sum(Fr37_chis >= 10**0.5)/sum(np.logical_or(Fr37_chis >= 10**0.5, Fr37_chis <= 10**-0.5))
Cond12 = sum(Fr37_chis <= 10**-0.5)/sum(np.logical_or(Fr37_chis >= 10**0.5, Fr37_chis <= 10**-0.5))
print('In the Bad Fits the fractions are:',
      'Over-fitting=%1.2f and under-fitting=%1.2f'%(Cond11, Cond12))

Total number of galaxies: 8439
The fraction of the removed galaxies depending on condition: Bad Fits=0.74, No-AGN=0.14 and Bad SFR=0.31
In the Bad Fits the fractions are: Over-fitting=0.65 and under-fitting=0.35


Here we count the classification types for the different CIGALE AGN models (To be used in Table 3). We read the VCV_SMB_otype.txt file to obtain the classifications and change the id in CIGALE models to a string format. Then, we join the tables and count the classifications.

In [20]:
Sample = Table.read('../Data/Final/VCV_SMB_otype.txt',
                    format='ascii').to_pandas()
SClean['main_id'] = np.array(SClean['id'], dtype='U30')
FClean['main_id'] = np.array(FClean['id'], dtype='U30')
SClean37['main_id'] = np.array(SClean37['id'], dtype='U30')
FClean37['main_id'] = np.array(FClean37['id'], dtype='U30')

In [21]:
SClass = SClean.to_pandas().join(Sample.set_index('main_id'), on='main_id')
FClass = FClean.to_pandas().join(Sample.set_index('main_id'), on='main_id')
S37Class = SClean37.to_pandas().join(Sample.set_index('main_id'), on='main_id')
F37Class = FClean37.to_pandas().join(Sample.set_index('main_id'), on='main_id')

##### SKIRTOR

In [22]:
print('Classified Seyfert galaxies:',
      sum(SClass.groupby('Sp').size()[['S1', 'S2', 'S']]))
print('Intermediate classifications',
      sum(SClass.groupby('Sp').size()[['S1.0', 'S1.2', 'S1.5',
                                       'S1.8', 'S1.9']]))
print('Other classifications',
      sum(SClass.groupby('Sp').size()[['S1h', 'S1i', 'S1n0', 'S1n2',
                                       'S1n5', 'S3', 'S3b', 'S?']]))
SClass.groupby('Sp').size()

Classified Seyfer galaxies: 7479
Intermediate classifications 511
Other classifications 70


Sp
S         30
S1      5975
S1.0     104
S1.2      94
S1.5     205
S1.8      66
S1.9      42
S1h       18
S1i        2
S1n0       5
S1n2       3
S1n5       5
S2      1474
S3        24
S3b        1
S?        12
dtype: int64

In [23]:
SClass.groupby('otype_txt').size()

otype_txt
Sy1    6389
Sy2    1626
SyG      45
dtype: int64

##### Fritz

In [24]:
print('Classified Seyfert galaxies:',
      sum(FClass.groupby('Sp').size()[['S1', 'S2', 'S']]))
print('Intermediate classifications',
      sum(FClass.groupby('Sp').size()[['S1.0', 'S1.2', 'S1.5',
                                       'S1.8', 'S1.9']]))
print('Other classifications',
      sum(FClass.groupby('Sp').size()[['S1h', 'S1i', 'S1n0', 'S1n2',
                                       'S1n5', 'S3', 'S3b', 'S?']]))
FClass.groupby('Sp').size()

Classified Seyfer galaxies: 7980
Intermediate classifications 492
Other classifications 65


Sp
S         29
S1      6438
S1.0     101
S1.2      88
S1.5     195
S1.8      66
S1.9      42
S1h       12
S1i        1
S1n0       4
S1n2       5
S1n5       5
S2      1513
S3        23
S3b        1
S?        14
dtype: int64

In [25]:
FClass.groupby('otype_txt').size()

otype_txt
Sy1    6821
Sy2    1673
SyG      43
dtype: int64

##### SKIRTOR with 30/70 angles

In [26]:
print('Classified Seyfert galaxies:',
      sum(S37Class.groupby('Sp').size()[['S1', 'S2', 'S']]))
print('Intermediate classifications',
      sum(S37Class.groupby('Sp').size()[['S1.0', 'S1.2', 'S1.5',
                                         'S1.8', 'S1.9']]))
print('Other classifications',
      sum(S37Class.groupby('Sp').size()[['S1h', 'S1i', 'S1n0', 'S1n2',
                                         'S1n5', 'S3', 'S3b', 'S?']]))
S37Class.groupby('Sp').size()

Classified Seyfer galaxies: 7540
Intermediate classifications 496
Other classifications 60


Sp
S         30
S1      6104
S1.0     100
S1.2      91
S1.5     198
S1.8      65
S1.9      42
S1h       14
S1i        2
S1n0       3
S1n2       3
S1n5       4
S2      1406
S3        19
S3b        1
S?        14
dtype: int64

In [27]:
S37Class.groupby('otype_txt').size()

otype_txt
Sy1    6492
Sy2    1561
SyG      43
dtype: int64

##### Fritz with 30/70 angles

In [28]:
print('Classified Seyfert galaxies:',
      sum(F37Class.groupby('Sp').size()[['S1', 'S2', 'S']]))
print('Intermediate classifications',
      sum(F37Class.groupby('Sp').size()[['S1.0', 'S1.2', 'S1.5',
                                         'S1.8', 'S1.9']]))
print('Other classifications',
      sum(F37Class.groupby('Sp').size()[['S1h', 'S1i', 'S1n0', 'S1n2',
                                         'S1n5', 'S3', 'S?']]))
F37Class.groupby('Sp').size()

Classified Seyfer galaxies: 7898
Intermediate classifications 482
Other classifications 59


Sp
S         30
S1      6494
S1.0      95
S1.2      85
S1.5     196
S1.8      64
S1.9      42
S1h       11
S1i        2
S1n0       4
S1n2       5
S1n5       5
S2      1374
S3        19
S?        13
dtype: int64

In [29]:
F37Class.groupby('otype_txt').size()

otype_txt
Sy1    6864
Sy2    1533
SyG      42
dtype: int64

##### Notebook info

In [30]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy,pandas,scipy

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.25.1.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.2
pandas : 1.2.0
scipy  : 1.6.0

matplotlib: 3.2.2
json      : 2.0.9
sys       : 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy     : 1.19.5
re        : 2.2.1
logging   : 0.5.1.2
autopep8  : 1.5.7

Watermark: 2.1.0

